In [ ]:
!apt-get update
!apt-get install -y \
    libatk1.0-0 \
    libatk-bridge2.0-0 \
    libcups2 \
    libxkbcommon0 \
    libatspi2.0-0 \
    libgtk-3-0 \
    libdbus-glib-1-2 \
    libnss3 \
    libxcomposite1 \
    libxrandr2 \
    libxdamage1 \
    libxfixes3 \
    libpango-1.0-0 \
    libpangocairo-1.0-0 \
    libcairo2 \
    libx11-xcb1 \
    libxcb1 \
    libxext6 \
    libxi6 \
    libxtst6
!pip install playwright
!playwright install chromium


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,637 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,289 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,592 kB]
Get

In [ ]:
!pip install playwright nest_asyncio sentence-transformers
!playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 21.2 MB/s eta 0:00:00
(node:1117) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [] 0% 0.0s164.7 MiB [] 0% 21.8s164.7 MiB [] 0% 10.6s164.7 MiB [] 0% 5.9s164.7 MiB [] 2% 3.1s164.7 MiB [] 3% 2.4s164.7 MiB [] 5% 2.1s164.7 MiB [] 6% 1.8s164.7 MiB [] 8% 1.7s164.7 MiB [] 9% 1.6s164.7 MiB [] 10% 1.5s164.7 MiB [] 12% 1.4s164.7 MiB [] 13% 1.4s164.7 MiB [] 14% 1.3s164.7 MiB [] 15% 1.3s164.7 MiB [] 17% 1.3s164.7 MiB [] 18% 1.2s164.7 MiB [] 19% 1.2s164.7 MiB [] 21% 1.2s164.7 MiB [] 22% 1.1s164.7 MiB [] 24% 1.0s164.7 MiB [] 26% 1.0s164.7 MiB [] 28% 0.9s164.7 MiB [] 30% 0.9s164.7 MiB [] 31% 0.9s164.7 MiB [] 33% 0.8s164.7 MiB [] 35% 0.8s164.7 MiB [] 37% 0.7s164.7 MiB [] 39% 0.7s164.7 MiB [] 41% 0.7

0bservation for Intelligent Book Pricing System (15 Books)
This program scrapes book data from BooksToScrape.com, identifies the correct book using Google Books API, fetches competitor prices from BooksRun API, and finally computes a smart selling price using a discount strategy.

 Key Observations
The system collects 15 book entries automatically by browsing page-by-page.

For each book, it retrieves:

Title

UPC code

Listed price from BooksToScrape

The Google Books API is used to match titles and extract:

ISBN-10 / ISBN-13

Author name

BooksRun API returns real-time competitor selling prices.

A discount percentage is chosen based on competitor price:

≥ ₹50 ⟶ 20% off

≥ ₹30 ⟶ 15% off

≥ ₹15 ⟶ 10% off

Otherwise ⟶ 5% off

The final recommended price is saved in:

CSV file

JSON file

Only books that have valid ISBN & competitor price are selected.

Progress is printed live (1st book, 2nd book, etc.).




In [2]:
import requests
import pandas as pd
import re
from pathlib import Path
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer, util
from urllib.parse import urljoin

BOOKSRUN_KEY = "vmslc6zs09uikpx6mhwr"

EXPORT_DIR = Path("output_15_books")
EXPORT_DIR.mkdir(exist_ok=True)

MODEL = SentenceTransformer("all-MiniLM-L6-v2")
BASE_URL = "https://books.toscrape.com/"

class IntelligentPricingSystem15Books:

    def __init__(self, api_key):
        self.api_key = api_key
        self.records = []

    def resolve_isbn(self, title):
        base_vec = MODEL.encode(title)
        url = f"https://www.googleapis.com/books/v1/volumes?q=intitle:{title}"

        try:
            data = requests.get(url, timeout=10).json()
            if "items" not in data:
                return None, None, None, None

            for item in data["items"]:
                info = item.get("volumeInfo", {})
                candidate_title = info.get("title", "")

                cand_vec = MODEL.encode(candidate_title)
                score = util.cos_sim(base_vec, cand_vec).item()

                if score < 0.70:
                    continue

                ids = info.get("industryIdentifiers", [])
                isbn10 = next((x["identifier"] for x in ids if x["type"] == "ISBN_10"), None)
                isbn13 = next((x["identifier"] for x in ids if x["type"] == "ISBN_13"), None)
                author = ", ".join(info.get("authors", ["Unknown"]))

                return isbn10, isbn13, candidate_title, author
        except:
            pass

        return None, None, None, None

    def booksrun_price(self, isbn):
        url = f"https://booksrun.com/api/v3/price/buy/{isbn}?key={self.api_key}"
        try:
            res = requests.get(url, timeout=10).json()
            offers = res.get("result", {}).get("offers", {})

            price = offers.get("booksrun", {}).get("used", {}).get("price")
            if price and price != "none":
                return float(price)

            market = offers.get("marketplace", [])
            prices = [
                float(x["used"]["price"])
                for x in market
                if x.get("used", {}).get("price") not in (None, "none")
            ]
            return min(prices) if prices else 0.0
        except:
            return 0.0

    def execute(self, required_books=15):
        collected = 0
        page_no = 1

        while collected < required_books:
            page_url = (
                urljoin(BASE_URL, "catalogue/page-1.html")
                if page_no == 1
                else urljoin(BASE_URL, f"catalogue/page-{page_no}.html")
            )

            res = requests.get(page_url)
            soup = BeautifulSoup(res.text, "html.parser")
            books = soup.select(".product_pod")

            for book in books:
                if collected >= required_books:
                    break

                detail_url = urljoin(page_url, book.h3.a["href"])
                detail_page = requests.get(detail_url)
                detail_soup = BeautifulSoup(detail_page.text, "html.parser")

                title = detail_soup.h1.text
                price_text = detail_soup.select_one(".price_color").text
                our_price = float(re.sub(r"[^\d.]", "", price_text))

                upc = ""
                for row in detail_soup.select("table.table-striped tr"):
                    if row.th.text == "UPC":
                        upc = row.td.text
                        break

                isbn10, isbn13, clean_title, author = self.resolve_isbn(title)
                if not isbn13:
                    continue

                competitor_price = self.booksrun_price(isbn13)
                if competitor_price <= 0:
                    continue

                if competitor_price >= 50:
                    discount = 0.20
                elif competitor_price >= 30:
                    discount = 0.15
                elif competitor_price >= 15:
                    discount = 0.10
                else:
                    discount = 0.05

                final_price = round(competitor_price * (1 - discount), 2)

                self.records.append({
                    "UPC": upc,
                    "ISBN-10": isbn10,
                    "ISBN-13": isbn13,
                    "Book Title": clean_title,
                    "Author": author,
                    "Source Price": our_price,
                    "Competitor Price": competitor_price,
                    "Discount Percent": int(discount * 100),
                    "Final Price": final_price
                })

                collected += 1
                print(f"{collected}. {clean_title}")

            page_no += 1

        df = pd.DataFrame(self.records)
        df.to_csv(EXPORT_DIR / "ai_pricing_15_books.csv", index=False)
        df.to_json(EXPORT_DIR / "ai_pricing_15_books.json", indent=4)

        return df


engine = IntelligentPricingSystem15Books(BOOKSRUN_KEY)
final_df = engine.execute(required_books=15)
print(final_df)


1. The Dirty Little Secrets of Getting Your Dream Job
2. orange: The Complete Collection 1
3. Lumberjanes Vol. 1
4. Thomas Jefferson and the Tripoli Pirates
5. The Regional Office is Under Attack!
6. The Murder of Roger Ackroyd
7. The Matchmaker's Playbook
8. The 10% Entrepreneur
9. Redeeming Love
10. Wild Swans
11. The Star-Touched Queen
12. The Marriage of Opposites
13. The Immortal Life of Henrietta Lacks
14. The Guilty: a Will Robie Novel 4
15. The Bane Chronicles
                 UPC     ISBN-10        ISBN-13  \
0   2597b5a345f45e1b  1942872763  9781942872764   
1   11809b1d323b3057  1626923027  9781626923027   
2   b6ee99bcf06fc91f  1608866874  9781608866878   
3   de3f25248ba0789f  0143131834  9780143131830   
4   0651062381f0636a  1594632413  9781594632419   
5   c7b5183f4d1d4efe  0425200477  9780425200476   
6   08db62f24627e71f  1503934489  9781503934481   
7   56e4f9eab2e8e674  1591848091  9781591848097   
8   b53db2271c9624f7  1590525132  9781590525135   
9   3fb15ace50d61